# Práctico 5: Cliff Walking

## Dependencias

In [1]:
import gymnasium
import numpy as np
import random

In [2]:
# Cambiar a rgb_array para entrenar/testear
env = gymnasium.make("CliffWalking-v0", render_mode="human")

Obtener la cantidad de estados y acciones

In [3]:
actions = env.action_space.n
states = env.observation_space.n

Inicialización de la tabla Q

In [ ]:
Q = np.zeros((states, actions))
Q

Obtención de la acción a partir de la tabla Q

In [5]:
def optimal_policy(state, Q):
    action = np.argmax(Q[state])
    return action

Epsilon-Greedy Policy

In [6]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = env.action_space.sample()
        print('explore')
    # exploit
    else:
        action = np.argmax(Q[state])
        print('exploit')
        
    return action

Ejemplo de episodio 

In [ ]:
obs,_ = env.reset()
print(obs)
done = False
total_reward = 0
step_count = 0
while not done:
    state = obs
    action = epsilon_greedy_policy(state, Q, 0.5)
    obs, reward, done, _, _ = env.step(action)
    total_reward += reward
    step_count += 1
    env.render()
print('total_reward', total_reward)
print('total_steps', step_count)